In [1]:
import psycopg2

In [2]:
class backend_official():
    def __init__(self,user,password,host,port,database):
        #self.connection = None
        #self.cursor = None
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.database = database
        
    def connect_to_database(self):
    try:
        connection = psycopg2.connect(user=self.user,
                                     password=self.password,
                                     host=self.host,
                                     port=self.port,
                                     database=self.database)
        cursor = connection.cursor()
        #print(connection.get_dsn_parameters(),"\n")

        cursor.execute("SELECT version();")
        record = cursor.fetchone()
        #print("You are connected to -",record,"\n")
        return connection
    except (Exception,psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
    
    def insert_profile(self,profile):

        connection = self.connect_to_database()
        cursor = connection.cursor()
        try:
            cursor.execute("INSERT INTO profile(name,email,referral,phone,password,age) VALUES(%s,%s,%s,%s,%s,%s)",(profile["name"],profile["email"],profile["referral"],profile["phone"],profile["password"],profile["age"]))
            connection.commit()
            return True
        except (Exception,psycopg2.Error) as error:
            return False
        finally:
            self.close_connection(connection,cursor)
    def close_connection(self,connection,cursor):
    if (connection):
        cursor.close()
        connection.close()